In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer


In [3]:
ca = pd.read_csv('dogma5.csv')



In [4]:
X = ca.drop(columns=[ca.columns[0], ca.columns[-1]])  # Признаки (все, кроме первого и последнего столбца)
y_class = ca.iloc[:, 0]  # Первая целевая переменная (покупка или нет)
y_reg = ca.iloc[:, -1]

In [5]:
print(X.isnull().sum())  # Количество пропусков в признаках
print(y_class.isnull().sum())  # Количество пропусков в первой целевой переменной
print(y_reg.isnull().sum()) 

0.0.4                 4865
1.0                    540
1.0.1                    0
0.0.10                4865
1.0.2                  541
                      ... 
274.6188739789965        0
271.6085180863477        0
273.10719564371834       0
7062679.317856605        0
6920071.582174959        0
Length: 72, dtype: int64
0
0


In [6]:
imputer = SimpleImputer(strategy='mean')
X_filled = imputer.fit_transform(X)

In [7]:
y_class = y_class.map({'yes': 1, 'no': 0})

In [8]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_filled)


In [9]:
ca = ca.dropna(subset=[ca.columns[0], ca.columns[-1]])

In [10]:
y_class = y_class.values

In [11]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_filled)  # Еще раз масштабируем признаки

# Проверим, что данных теперь нет пропусков
print(pd.isnull(X_scaled).sum())  # Для признаков

0


In [12]:
ca = ca.dropna(subset=[ca.columns[0], ca.columns[-1]])

# Переприсвоим X_scaled и y_class
X_scaled = ca.drop([ca.columns[0], ca.columns[-1]], axis=1).values
y_class = ca[ca.columns[0]].values  # Или, если первый столбец - это 
print("Количество пропусков в y_class:", pd.isnull(y_class).sum())

Количество пропусков в y_class: 0


In [13]:
ca_cleaned = ca.dropna(axis=1)

# Преобразуем X и y
X_cleaned = ca_cleaned.drop([ca_cleaned.columns[0], ca_cleaned.columns[-1]], axis=1).values
y_class_cleaned = ca_cleaned[ca_cleaned.columns[0]].values  # Целевая переменная - первый столбец

# Проверим, что пропусков больше нет
print("Количество пропусков в X_cleaned:", pd.isnull(X_cleaned).sum())
print("Количество пропусков в y_class_cleaned:", pd.isnull(y_class_cleaned).sum())

# Обучим модель


Количество пропусков в X_cleaned: 0
Количество пропусков в y_class_cleaned: 0


In [ ]:
model_class = LogisticRegression(max_iter=100, solver='liblinear', class_weight='balanced')
model_class.fit(X_cleaned, y_class_cleaned)

In [ ]:
y_pred_class = model_class.predict(X_scaled)

In [ ]:
accuracy = accuracy_score(y_class, y_pred_class)
print(f'Accuracy (Classification Model): {accuracy:.4f}')

In [ ]:
model_reg = LinearRegression()
model_reg.fit(X_scaled, y_reg)


In [ ]:
y_pred_reg = model_reg.predict(X_scaled)

In [ ]:
mse = mean_squared_error(y_reg, y_pred_reg)
r2 = r2_score(y_reg, y_pred_reg)
print(f'MSE (Regression Model): {mse:.4f}')
print(f'R² (Regression Model): {r2:.4f}')